In [12]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
#from langchain.vectorstores import Pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os


In [13]:
# trỏ địa chỉ đến api_key
#pc = Pinecone(api_key="0dcca609-9f73-471b-9e57-882afb68c546")

# tạo index (đã tạo rồi nên skip)
#pc.create_index(name="cooking-chatbot", dimension=384, spec=ServerlessSpec(cloud='aws', region='us-east-1'))
#index = pc.Index("cooking-chatbot")

In [14]:
PINECONE_API_KEY = '0dcca609-9f73-471b-9e57-882afb68c546'

In [15]:
pc = Pinecone(api_key=PINECONE_API_KEY)
#pc.list_indexes().names() # check if index exsist
index_name='cooking-chatbot'
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'cooking-chatbot': {'vector_count': 13872}},
 'total_vector_count': 13872}

In [16]:
def load_data(data):
    loader = DirectoryLoader(data, glob='*.pdf', loader_cls=PyPDFLoader)

    documents = loader.load()
    return documents

In [17]:
extracted_data = load_data("data")

Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 560 0 (offset 0)


In [18]:
# create text chucks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [19]:
text_chunks = text_split(extracted_data)
print("length of text chunks: ", len(text_chunks))

length of text chunks:  4624


In [20]:
def download_huggingface_embedding():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings             

In [21]:
embeddings = download_huggingface_embedding()

c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [22]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [23]:
query_result = embeddings.embed_query("Hello world")
print('Length', len(query_result))

Length 384


In [24]:
query_result

[-0.03447726368904114,
 0.031023245304822922,
 0.006734971422702074,
 0.02610901929438114,
 -0.03936200216412544,
 -0.16030250489711761,
 0.06692396849393845,
 -0.006441470701247454,
 -0.04745050147175789,
 0.014758853241801262,
 0.070875383913517,
 0.05552757531404495,
 0.019193332642316818,
 -0.026251336559653282,
 -0.01010945625603199,
 -0.026940515264868736,
 0.022307420149445534,
 -0.022226683795452118,
 -0.1496926099061966,
 -0.01749308593571186,
 0.007676285691559315,
 0.054352324455976486,
 0.0032545297872275114,
 0.031725965440273285,
 -0.0846213847398758,
 -0.02940598875284195,
 0.05159568041563034,
 0.048124056309461594,
 -0.003314773552119732,
 -0.05827918276190758,
 0.04196930304169655,
 0.02221071720123291,
 0.12818880379199982,
 -0.022338956594467163,
 -0.011656234040856361,
 0.06292833387851715,
 -0.03287616744637489,
 -0.09122607111930847,
 -0.03117538057267666,
 0.052699584513902664,
 0.047034814953804016,
 -0.08420303463935852,
 -0.03005615621805191,
 -0.020744813606

In [25]:
docs_chunks = [t.page_content for t in text_chunks]

vectorstore = PineconeVectorStore(index=index, pinecone_api_key=PINECONE_API_KEY, embedding=embeddings, namespace='cooking-chatbot',index_name='cooking-chatbot')

vectorstore.add_texts(texts=[t.page_content for t in text_chunks])


['62d35728-2760-4716-8ce0-1d18d06553b8',
 '5069e391-260a-4f5c-859f-6d64356c9a4f',
 '49cb5fea-b719-4cba-aa91-3ea798163223',
 'a1cc8b1b-c743-4210-b522-54bb3ce1ab85',
 'dc6016ea-ce28-41ac-b750-d8951e77839c',
 'f6575f96-5eca-4e65-8b1a-2debd718d007',
 'd220e1e3-32aa-45f7-b4c3-96f3b675bb3b',
 'f827e181-5bf0-4ed1-b392-e56ccf29860a',
 'b6554ddd-6f89-4316-b0dc-4042e7d7957f',
 '1604ec5d-5137-483c-b713-6f5ff9283277',
 '608232a5-7c24-409f-990b-74bc47b568be',
 '69325d5f-f5bf-4856-9324-aeb7ea99f37c',
 'c8fe2ef2-7cf0-4769-90d3-420cdd63491c',
 '47aa8651-e5c3-4a10-a5f0-7c6a008aa20f',
 '7da11410-20c4-4aa8-9096-baa4cb1c7abf',
 'b58275f5-7118-4c6c-b178-873629849d4f',
 '39525d98-a707-49ff-9ffd-62eecfbf9561',
 '4989ec6a-7b94-4c50-8923-a1357c09e59d',
 '2ecb42f2-7127-4180-95db-f8a056b0de8a',
 '02952f6a-2007-4d82-889b-d162b24ef6b9',
 'c71ff2fb-40e0-4c20-98bc-3e63feaa12a3',
 '68079cf4-c427-49c5-abb2-2e631bb0f02b',
 '88bfbe8e-b080-481d-b447-6bc0ab58eeaf',
 'bc38aa5a-17af-49f1-aafb-288ad3175fba',
 '72ce1157-9fc0-

In [26]:
vectorstore_existing = PineconeVectorStore(index=index, pinecone_api_key=PINECONE_API_KEY, embedding=embeddings, namespace='cooking-chatbot', index_name='cooking-chatbot')

query = "what is pasta"

docs = vectorstore_existing.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='This\t\r \xa0can\t\r \xa0be\t\r \xa0enjoyed\t\r \xa0as\t\r \xa0a\t\r \xa0simple\t\r \xa0tomato\t\r \xa0pasta,\t\r \xa0or\t\r \xa0with\t\r \xa0cheese\t\r \xa0or\t\r \xa0tuna,\t\r \xa0or\t\r \xa0both\t\r \xa0:)'), Document(page_content='This\t\r \xa0can\t\r \xa0be\t\r \xa0enjoyed\t\r \xa0as\t\r \xa0a\t\r \xa0simple\t\r \xa0tomato\t\r \xa0pasta,\t\r \xa0or\t\r \xa0with\t\r \xa0cheese\t\r \xa0or\t\r \xa0tuna,\t\r \xa0or\t\r \xa0both\t\r \xa0:)'), Document(page_content='This\t\r \xa0can\t\r \xa0be\t\r \xa0enjoyed\t\r \xa0as\t\r \xa0a\t\r \xa0simple\t\r \xa0tomato\t\r \xa0pasta,\t\r \xa0or\t\r \xa0with\t\r \xa0cheese\t\r \xa0or\t\r \xa0tuna,\t\r \xa0or\t\r \xa0both\t\r \xa0:)')]


In [27]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [28]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [29]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [13]:
# Creating a retriever from vectorstore_existing
retriever = vectorstore_existing.as_retriever(search_kwargs={'k': 2})

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs
)

NameError: name 'vectorstore_existing' is not defined

In [32]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])